## Part 1: Chain-of-Thought (CoT) 搭配 Prompting 工程

In [1]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')



In [ ]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }

  payload["seed"] = 0 # 為了重現一樣的結果，這裡固定了 seed

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

### 一個簡單的思考題目

Q: 我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？

In [4]:
# 出處: https://promptingguide.azurewebsites.net/techniques/cot
user_message = """
我去7-11買了10個香蕉。我給了弟弟2個香蕉，又給修理工2個香條。然後自己1個。另外隔壁老王給我3個，我還剩下多少個香蕉？
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.2, model="gpt-3.5-turbo")
print(response)


10 - 2 - 2 + 3 = 9

你還剩下2個香蕉。


❌❌ GPT-3.5 竟然算錯了!!

## Few-shot CoT (給範例)

那針對比較笨的模型，可以如何增強推理能力呢? 

## Chain of Thought (CoT) 思考過程:
Chain of Thought (CoT)是一種被用於增強大型語言模型（比如GPT系列）解決問題和推理任務能力的方法。這種方法通過鼓勵模型採用一種分步驟的、邏輯推理的方式來處理複雜的問題，從而能更好地理解和回答這些問題。

中心思想: 先想清楚再回答!

In [5]:
user_message = """
Q: 我去市場買了6個香蕉，給了朋友3個香蕉，我還剩下多少個?
A:
  1. 我一開始有6個
  2. 給了朋友3個，所以剩下 6-3=3個香蕉
  3. 最後剩下3個香蕉

Q: 我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
A:
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.2, model="gpt-3.5-turbo")
print(response)

1. 我一開始有10個蘋果
2. 給了鄰居2個蘋果，剩下 10-2=8個蘋果
3. 又給了修理工2個蘋果，剩下 8-2=6個蘋果
4. 再買了5個蘋果，總共有 6+5=11個蘋果
5. 吃了1個蘋果，剩下 11-1=10個蘋果
6. 最後剩下10個蘋果。


## Zero-shot CoT (讓模型自己想步驟)

- 標準咒語 </br>
“Let's think step by step”（讓我們逐步思考）是一種用於指引模型按照連續的步驟來思考問題的方式。這種方法鼓勵模型不直接跳到最終答案，而是展示出達到答案的過程，這有助於模型更好地處理複雜的推理任務。

- 最新咒語 </br>
根據2023年9月3日發布於arXiv的論文，一種更新的咒語“Take a deep breath and work on this problem step-by-step.”（深呼吸，然後逐步解決這個問題）被提出，旨在進一步增強模型逐步推理的能力。這個咒語不僅鼓勵模型按部就班地解決問題，還引導模型在開始逐步推理之前，先行“深呼吸”，這可能象徵著讓模型在開始解決問題前進入一種更加準備好的狀態。

In [6]:
user_message = """
我去市場買了10個蘋果。我給了鄰居2個蘋果，又給修理工2個蘋果。之後，我又去買了5個蘋果，然後吃了1個。我還剩下多少個蘋果？
Let's think step by step
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.2)
print(response)

1. 我一開始買了10個蘋果
2. 我給了鄰居2個蘋果，還剩下8個蘋果
3. 我又給修理工2個蘋果，還剩下6個蘋果
4. 我又去買了5個蘋果，總共擁有11個蘋果
5. 我吃了1個蘋果，還剩下10個蘋果

所以，我還剩下10個蘋果。


### case1 : 有沒有 「 一步一步思考很重要 」，prompt 優化

In [7]:
# 出處: https://platform.openai.com/docs/guides/gpt-best-practices
# 失敗問法:
system_promot = "請判斷學生的解答是否正確"
user_prompt = """"
問題描述：

我正在建造一個太陽能發電系統，需要財務計算:
- 土地成本為每平方英尺100美元
- 我可以用每平方英尺250美元的價格購買太陽能板
- 我簽訂了一份保養合約，每年將花費我固定的10萬美元，以及每平方英尺10美元的額外費用。

請問隨著平方英尺數量的增加，第一年運營的總成本是多少？

學生的解答:

令 x 為安裝面積的平方英尺。

1. 土地成本：100x
2. 太陽能板成本：250x
3. 維護成本：100,000 + 100x

總成本: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

messages = [
  { "role": "system", "content": system_promot },
  { "role": "user", "content": user_prompt }
]

response = get_completion(messages, temperature=0.2, model="gpt-4-0613")
print(response)

學生的解答是正確的。他們正確地將所有的成本因素加在一起，並將其表示為安裝面積的函數。


In [8]:
# 成功問法:
system_promot = """請依照以下步驟一步一步思考:

1. 請先自己解決問題，算出最終答案
2. 比較你的答案和學生的解決方案
3. 最後判斷學生的解決方案是否正確

在自己解決問題之前，不要決定學生的解決方案是否正確
"""

user_prompt = """"
問題描述：

我正在建造一個太陽能發電系統，需要財務計算:
- 土地成本為每平方英尺100美元
- 我可以用每平方英尺250美元的價格購買太陽能板
- 我簽訂了一份保養合約，每年將花費我固定的10萬美元，以及每平方英尺10美元的額外費用。

請問隨著平方英尺數量的增加，第一年運營的總成本是多少？

學生的解答:

令 x 為安裝面積的平方英尺。

1. 土地成本：100x
2. 太陽能板成本：250x
3. 維護成本：100,000 + 100x

總成本: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

messages = [
  { "role": "system", "content": system_promot },
  { "role": "user", "content": user_prompt }
]

response = get_completion(messages, temperature=0.2, model="gpt-4-0613")
print(response)

1. 我的解決方案：

我會用同樣的方式來解決這個問題。首先，我會將每個成本項目與平方英尺數量相乘，然後將所有的成本加起來。

土地成本：100x
太陽能板成本：250x
維護成本：100,000 + 10x

總成本: 100x + 250x + 100,000 + 10x = 360x + 100,000

2. 比較我的答案和學生的解決方案：

學生在計算維護成本時，將每平方英尺的額外費用誤寫為100美元，實際上應該是10美元。

3. 學生的解決方案是否正確？

學生的解決方案不正確，因為他在計算維護成本時出現了錯誤。正確的總成本應該是360x + 100,000，而不是450x + 100,000。


### case2 : 跟模型強調要展開思考步驟 => 5 why 思考法 (很重要連續問 5 次)

In [12]:
# 案例參考出處: https://iamhlb.notion.site/AI-3035cc7781a948dbaed25d2ff59c84fb
user_message = """
你是一個廣告創意大師，請根據以下商品和步驟，一步一步探索用戶心理:

商品: vision pro 3D眼鏡

Step 1: 為何人們需要買這個商品?
Step 2: 根據上一步的回答，繼續深究為什麼?
Step 3: 根據上一步的回答，繼續深究為什麼?
Step 4: 根據上一步的回答，繼續深究為什麼?
Step 5: 根據上一步的回答，繼續深究為什麼?

最後，請根據上述的原因，輸出一個購買的最佳理由文案。
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.7)
print(response)

Step 1: 人們需要買這個商品是因為他們想要享受更震撼的3D視覺效果。

Step 2: 人們想要享受更震撼的3D視覺效果是因為他們希望在觀影、遊戲或其他娛樂活動中獲得更加身臨其境的感受。

Step 3: 人們希望在觀影、遊戲或其他娛樂活動中獲得更加身臨其境的感受是因為他們想要擺脫疲勞和壓力，尋找一種放鬆身心的方式。

Step 4: 人們想要擺脫疲勞和壓力，尋找一種放鬆身心的方式是因為他們珍惜自己的休閒時光，希望在忙碌的生活中找到一絲樂趣和放鬆。

Step 5: 人們珍惜自己的休閒時光，希望在忙碌的生活中找到一絲樂趣和放鬆，所以他們需要一款優質的3D眼鏡，讓他們在家中就能享受到戲院般的視覺體驗。

最佳理由文案: 「讓您在家中也能享受戲院般的震撼視覺效果，放鬆身心，擺脫疲勞和壓力，尋找一絲樂趣和放鬆，Vision Pro 3D眼鏡，帶您進入全新的視覺體驗世界。」


In [13]:
target = "生活備感壓力的年輕上班族"
product = "麥當勞推出史上最貴的龍蝦堡"

prompt = f"""你是一個善於挖掘消費者內心洞察的廣告大師，請根據以下資訊:

目標受眾: {target}
商品: {product}

用以下步驟進行思考回答，必須要符合目標受眾喜好，且符合產品要傳遞的價值:

1. 根據你的專業，回答目標受眾的 "內在心理動機"，此為目標受眾的狀態。針對這個狀態，請回答為什麼，此為 "理由"。回答格式為：「我現在(狀態)，是因為我想要(理由)」
2. 根據第一個步驟的敘述，以目標受眾第一人視角，說出目標受眾不敢說出、但希望被了解的「一句話」，這句話必須讓人引起極為激烈的情緒，無論是開心、悲傷、憤怒、感動都可以
3. 根據第一個步驟的敘述，用一個具體物品名詞來詮釋，但不能與產品是同性質的物品
4. 根據第一個步驟的敘述，用關鍵詞彙來譬喻或類比或聯想目標受眾，並寫下這段深刻的描述
5. 根據前述描述資訊，發想出一個極度吸引人的報名國際廣告獎使用的"廣告活動名稱"，必須使用到關鍵詞彙或關鍵詞彙的概念，必須讓評審一看到就產生極高的好奇心，並且會在社群上造成轟動
6. 請重複上述步驟1~5共五次，上一次的理由，就是下一次的狀態，以此類推

"""

result = get_completion([{"role": "user", "content": prompt }], temperature=0.7, model="gpt-4")
print(result)

# 此 CoT 有難度，因此 gpt-4 可以 CoT，但 gpt-3.5-turbo 不行辦不到
# gpt-3.5-turbo 得用 chaining


1. 我現在感覺生活壓力大，是因為我想要逃脫現實的煩惱和壓力。
2. 「我不希望每天都是沉重的壓力，我只希望有一天可以輕鬆自在。」
3. 高海拔的山峰
4. 就像在崇高的山峰之上，看著壓力如同山下的雲霧，而自己可以在山頂享受一份龍蝦堡的悠閒。
5. 廣告活動名稱：「山峰頂的龍蝦堡」

1. 我現在想要逃脫現實的煩惱和壓力，是因為我想要享受生活的美好時光。
2. 「我希望能有一個屬於我的空間，沒有壓力，沒有煩惱，只有享受和快樂。」
3. 夕陽下的沙灘
4. 就像在夕陽下的沙灘，享受著暖暖的陽光，吃著龍蝦堡，感受生活的美好。
5. 廣告活動名稱：「夕陽沙灘的龍蝦時刻」

1. 我現在想要享受生活的美好時光，是因為我想要對自己好一點。
2. 「我知道我值得更好的，我應該對自己好一點，讓自己能享受生活的美好。」
3. 精緻的甜點
4. 就像精緻的甜點，帶給我們甜蜜的滋味，龍蝦堡也能給我們帶來生活的美好。
5. 廣告活動名稱：「甜蜜的自我犒賞龍蝦堡」

1. 我現在想要對自己好一點，是因為我想要感受到生活的價值。
2. 「我希望能感受到生活的價值，讓自己知道，每一刻的努力都是值得的。」
3. 值得的獎牌
4. 就像獲得的獎牌，感受到自己努力的價值，龍蝦堡也能讓我們感受到生活的價值。
5. 廣告活動名稱：「生活獎牌：龍蝦堡」

1. 我現在想要感受到生活的價值，是因為我想要找回生活的熱情。
2. 「我希望能找回生活的熱情，讓自己每天都充滿動力，享受生活的每一刻。」
3. 熱情的火焰
4. 就像熱情的火焰，燃燒著生活的熱情，龍蝦堡也能讓我們找回生活的熱情。
5. 廣告活動名稱：「熱情的龍蝦堡」


## CoT 需要輸出過程，不能省略

語言模型本質就是詞語接龍，有步驟和沒步驟，結果差異非常大!!


In [17]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

最後的答案 x 是多少?

"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

Step 1: x = 100 + 1 = 101
Step 2: x = 101 + 10 = 111
Step 3: x = 111 - 1 = 110
Step 4: x = 110 * 2 = 220
Step 5: x = 220 - 20 = 200

最後的答案 x = 200.


### 不輸出步驟，只輸出答案，有可能會錯

In [15]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

不要輸出過程，只要回答最後 x 是多少?

"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0.5)
print(response)

最後 x = 190


### 先輸出答案，再輸出過程，還是會錯

In [16]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

請直接回答最後答案，然後再解釋說明是如何算出來的

"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

最後答案是 189。

解釋：
Step 1: x = 100 + 1 = 101
Step 2: x = 101 + 10 = 111
Step 3: x = 111 - 1 = 110
Step 4: x = 110 * 2 = 220
Step 5: x = 220 - 20 = 200

因此，最後答案是 200。


### 結論: 有推理過程會幫助思考，文字接龍效果才會好

# Part 2: CoT 延伸策略:

## Self-criticism 策略

在 AI 給出答案之後，再繼續追問一次請 AI 分析並改進答案


In [9]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

不要輸出過程，只要回答最後 x 是多少?

"""

messages = [
    {
        "role": "user",
        "content": user_message
    },
    {
        "role": "assistant",
        "content": "最後 x 是 189。" # 這裡是放剛剛 AI 的回答
    },
    {
        "role": "user",
        "content": 
        """
        #zh-tw
        1. Review your previous answer and find problems with your answer.
        2. Based on the problems you found, improve your answer.
        """
    },
]

response = get_completion(messages, temperature=0)
print(response)

對不起，我之前的答案有誤。讓我重新計算一次：

Step 1: x 加 1
100 + 1 = 101

Step 2: x 加 10
101 + 10 = 111

Step 3: x 減 1
111 - 1 = 110

Step 4: x 乘 2
110 * 2 = 220

Step 5: x 減 20
220 - 20 = 200

所以最後 x 是 200。感謝您的提醒。


## Generated Knowledge 策略

* https://learnprompting.org/docs/intermediate/generated_knowledge
* https://developer.nvidia.com/blog/how-to-get-better-outputs-from-your-large-language-model/ (Prompt with generated knowledge)

在生成最終回答之前，要求LLM 先生成關於問題的潛在有用資訊。可以改進最終答案，但還是可能會錯。



In [21]:
# 沒有給前提，AI 會回答不好

user_message = "請寫一篇關於台灣石虎的介紹文章，文章要有趣，大約100字"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

台灣石虎是台灣特有的貓科動物，外表酷似豹貓，但體型更為嬌小可愛。牠們喜歡在山區或林地中活動，主要以小型哺乳動物和昆蟲為食。由於棲息地遭到破壞和獵捕，台灣石虎已瀕臨絕種，成為台灣的保育焦點之一。讓我們一起努力保護這些可愛的生物，讓牠們在台灣的山林中繼續生存下去吧！


In [22]:
# 有前提，AI 會回答較好
user_message = "請先產生三條關於台灣石虎的結構化知識，然後再寫一篇介紹文章，文章要有趣，大約100字"

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

1. 台灣石虎是台灣特有的瀕危野生動物，屬於貓科動物。
2. 台灣石虎的身長約為60至80公分，尾巴長約為30至40公分。
3. 台灣石虎主要棲息在中低海拔山區的森林中，以小型哺乳動物和鳥類為食。

在台灣的山林中，有一種神祕的生物，它就是台灣石虎！這隻可愛的貓科動物，身形優雅，毛色漂亮。它們喜歡在樹林中覓食，捕捉小動物。然而，由於棲息地遭到破壞和人類活動的干擾，台灣石虎的數量逐漸減少，現在已經成為瀕危物種。讓我們一起努力保護這些可愛的生物，讓牠們在台灣的山林中繼續生存下去吧！


## 內心 OS 策略

不能省略輸出思考步驟，但我們又不想讓用戶看到中間過程。
可以用分隔號或是 XML, JSON 區隔，這樣就可以方便擷取出答案，不讓用戶看到中間思考過程。

In [10]:
user_message = """
請用以下計算數學計算，假設 x = 100

Step 1: x 加 1
Step 2: x 加 10
Step 3: x 減 1
Step 4: x 乘 2
Step 5: x 減 20

請將思考步驟放進 <thinking> </thinking> XML 標籤。
最後的答案 x 請放進 <answer> </answer> XML 標籤。
"""

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_completion(messages, temperature=0)
print(response)

<thinking>
Step 1: x = 100 + 1 = 101
Step 2: x = 101 + 10 = 111
Step 3: x = 111 - 1 = 110
Step 4: x = 110 * 2 = 220
Step 5: x = 220 - 20 = 200
</thinking>

<answer>
200
</answer>


## Self-Consistency (Ensembling) 此方法很棒!

 https://learnprompting.org/docs/intermediate/self_consistency

 產生多個 zero-shot CoT 推理路徑，然後用最一致的答案。
用此法溫度就不設定0了。

In [11]:
question = """
你好，

我在您的系統中發現了一個重大安全漏洞。 雖然它不是
易於使用，可以訪問所有用戶的數據。 我附加了
概念證明。 請盡快修復此問題。

桑尼


###
Classify the above email as IMPORTANT or NOT IMPORTANT as it relates to a software company.
Let's think step by step. 
思考過程步驟用 <thinking> </thinking> XML 標籤包裹起來，
然後最後答案用 <answer> </answer> XML 標籤包裹起來輸出。

"""

messages = [
    {
        "role": "user",
        "content": question
    }
]

for x in range(3):
  response = get_completion(messages, temperature=0.5, model="gpt-3.5-turbo")
  print(response)



<thinking>
1. 這封郵件提到了一個重大安全漏洞，這對於一家軟件公司來說是非常重要的信息。
2. 郵件中提到漏洞可能會影響所有用戶的數據，這可能導致嚴重的安全問題。
3. 郵件的內容似乎是在提醒公司修復問題，這對於保護客戶數據和公司聲譽是至關重要的。
4. 總的來說，這封郵件涉及到軟件公司的安全問題，因此可以被歸類為重要。

</thinking>
<answer>IMPORTANT</answer>
<thinking>
1. 這封郵件提到了一個重大安全漏洞，這對於一家軟件公司來說是非常重要的信息。
2. 安全漏洞可能會導致用戶數據泄露，這可能對公司的聲譽和客戶信任造成嚴重影響。
3. 郵件中提到了附加的概念證明，這表明寄件人已經做了一些調查和準備工作。
4. 修復安全漏洞是公司的責任，並且應該被視為一個緊急問題。

基於以上分析，我認為這封郵件對於一家軟件公司來說是非常重要的。
</thinking>

<answer>IMPORTANT</answer>
<thinking>
1. 這封郵件提到了一個重大安全漏洞，這對於一家軟件公司來說是非常重要的信息。
2. 雖然郵件中提到這個漏洞不是易於使用，但仍然存在風險。
3. 修復這個問題是至關重要的，以確保用戶數據的安全性。
4. 因此，這封郵件應該被視為重要的。
</thinking>

<answer>
IMPORTANT
</answer>


其實 OpenAI API 內建參數就可以輸出多個結果，只是很少人用，太費 tokens 了

https://platform.openai.com/docs/api-reference/chat/create

In [15]:
import requests
import json


def get_completion_n(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=2000, n=1):
    payload = {
        "model": model,
        "temperature": temperature,
        "messages": messages,
        "max_tokens": max_tokens,
        "n": n
    }
    headers = {
        "Authorization": f'Bearer {openai_api_key}',
        "Content-Type": "application/json"
    }
    response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, data=json.dumps(payload))
    obj = json.loads(response.text)
    if response.status_code == 200:
        return obj["choices"]
    else:
        return obj["error"]


n_results = 5  # 假设我们希望获得5个独立的结果
results = get_completion_n(messages, temperature=0.5, model="gpt-3.5-turbo", n=n_results)

if isinstance(results, list):
    for i, choice in enumerate(results):
        print(f"Result {i+1}: {choice['message']['content']}\n")
else:
    print("Error:", results)


Result 1: <thinking>
1. The email mentions a major security vulnerability in the system of a software company.
2. The vulnerability allows access to all user data.
3. The sender attached a proof of concept to demonstrate the vulnerability.
4. The sender requests a prompt fix for the issue.

Based on the above points, the email should be classified as IMPORTANT as it directly relates to the security of the software company's system.
</thinking>

<answer>IMPORTANT</answer>

Result 2: <thinking>
1. The email mentions a major security vulnerability in the system of the software company.
2. The vulnerability allows access to all user data, which is a serious issue.
3. The sender has attached a proof of concept to demonstrate the vulnerability.
4. The sender is urging the company to fix the issue as soon as possible.
5. Overall, the email is important as it highlights a critical security issue that needs immediate attention.
</thinking>

<answer>IMPORTANT</answer>

Result 3: <thinking>
1. Th

In [16]:
pp(results)

[{'index': 0,
  'message': {'role': 'assistant',
              'content': '<thinking>\n'
                         '1. The email mentions a major security vulnerability '
                         'in the system of a software company.\n'
                         '2. The vulnerability allows access to all user '
                         'data.\n'
                         '3. The sender attached a proof of concept to '
                         'demonstrate the vulnerability.\n'
                         '4. The sender requests a prompt fix for the issue.\n'
                         '\n'
                         'Based on the above points, the email should be '
                         'classified as IMPORTANT as it directly relates to '
                         "the security of the software company's system.\n"
                         '</thinking>\n'
                         '\n'
                         '<answer>IMPORTANT</answer>',
              'refusal': None},
  'logprobs': None,
  'f

# CoT = ( prompt1 + prompt2 + prompt3 ....) 將大問題拆解成小問題，然後一步一步解決

In [17]:
question = f"""
請將 result 中的結果進行總結 {results}

###
思考過程步驟用 <thinking> </thinking> XML 標籤包裹起來，
然後最後答案用 <answer> </answer> XML 標籤包裹起來輸出。

"""

messages = [
    {
        "role": "user",
        "content": question
    }
]


response = get_completion(messages, temperature=0.5, model="gpt-3.5-turbo")
print(response)



根據結果中的總結，所有郵件都提到了一個重大的安全漏洞，該漏洞允許訪問所有用戶數據，並且寄件人附上了一個概念證明來展示這個漏洞。寄件人要求軟件公司立即修復這個問題。根據這些信息，所有郵件都被歸類為「重要」，因為它們直接關係到軟件公司系統的安全性。


## Least to Most Prompting 策略

較複雜的問題，可以先拆解成子問題 (Decomposition) 再回答

* https://www.breezedeus.com/article/llm-prompt-l2m
* https://learnprompting.org/zh-Hans/docs/intermediate/least_to_most

來下個咒語是 Let's break down this problem

In [18]:
messages = [
    {"role": "user", "content": """

    Q: Amy在滑梯底端，爬到滑梯頂端需要4分鐘。她需要1分鐘才能滑下來。滑梯將在19分鐘後關閉。請問在關閉前他還可以滑幾次?
    
    ###
    Let's break down this problem: (#zh-tw)
     
    """
    }
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)

1. Amy需要4分鐘爬到滑梯頂端，然後1分鐘才能滑下來，所以每次滑梯的循環時間為4+1=5分鐘。
2. 滑梯將在19分鐘後關閉，所以在這段時間內，Amy可以滑梯的次數為19/5=3次，取整數部分。

所以在滑梯關閉前，Amy還可以滑3次。


In [20]:
messages = [
    {"role": "user", "content": """
     
Q: Amy在滑梯底端，爬到滑梯頂端需要4分鐘。她需要1分鐘才能滑下來。滑梯將在19分鐘後關閉。請問在關閉前他還可以滑幾次?

請用以下步驟一步一步思考:

1. 在回答這個問題之前，必須先解決哪些子問題?
2. 請將上述子問題，重新排序從簡單到困難
3. 請依序回答子問題
4. 總結以上，最後回答原始問題

"""}
]

result = get_completion(messages, model="gpt-3.5-turbo-1106")
print(result)


1. 子問題包括：Amy能夠滑幾次？她需要多久才能爬到滑梯頂端？她需要多久才能滑下來？滑梯將在多久後關閉？

2. 子問題重新排序如下：
   a. 滑梯將在多久後關閉？
   b. Amy需要多久才能滑下來？
   c. Amy需要多久才能爬到滑梯頂端？
   d. Amy能夠在滑梯關閉前滑幾次？

3. 回答子問題：
   a. 滑梯將在19分鐘後關閉。
   b. Amy需要1分鐘才能滑下來。
   c. Amy需要4分鐘才能爬到滑梯頂端。
   d. Amy在滑梯關閉前可以滑3次。

4. 總結：在滑梯關閉前，Amy還可以滑3次。


# 總結複習 
分而治之：</br>
將大問題拆分成小問題解決，再將小問題的解答組合成大問題的解答。

金字塔結構：</br>
先提出總論，然後逐步細化，最後給出具體例子或證據。

邏輯推理
- 大前提：一個普遍性的聲明或規則。
- 小前提：特定情況下的聲明。
- 結論：從大前提和小前提推導出的結論。

## CoT

* 給步驟增加模型思考時間
* 讓模型自己想步驟: Let's think step by step  

## LLM 原理

* CoT 需要輸出過程，不能省略
* CoT 需要先輸出思考過程，再輸出答案

## 各種 CoT 延伸策略

* Self-criticism 策略: 給 AI 再一輪機會，修正答案
* Generated Knowledge 策略: 先生成關於問題的潛在有用信息，再生成答案
* Take a Step Back 技巧: 先將原問題重新 rephrase 成更一般性的問題，引出一段相關知識，然後再回答用戶問題
* 內心獨白策略: 可拆開思考過程和答案輸出
* Self-Consistency (Ensembling)
* Least to Most Prompting 法: 拆解大問題
* Tree-of-Thought (ToT) 這額外補充在 colab 208
